# Import

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from numpy import log as ln

# DF

In [33]:
#First supergames.xlsx - for first super games
#second supergame.xlsx - for second super games
#MLE for K=5 for all data

In [34]:
data = pd.read_excel("All supergames.xlsx")

In [35]:
data.head()

,paper,treatment:,ID,supergame,round,coop,ocoop,roundone,aftercc,aftercd,afterdc,afterdd
0,Dal Bo 2005,18,327,1,1,defect,coop,0.0,NaN,NaN,NaN,NaN
1,Dal Bo 2005,18,327,1,2,defect,defect,NaN,NaN,NaN,0.0,NaN
2,Dal Bo 2005,18,327,2,1,defect,coop,0.0,NaN,NaN,NaN,NaN
3,Dal Bo 2005,18,327,2,2,defect,defect,NaN,NaN,NaN,0.0,NaN
4,Dal Bo 2005,18,327,2,3,defect,defect,NaN,NaN,NaN,NaN,0.0


# Define the Vectors

In [36]:
AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB = 0.0214,0.0214,0.0214,0.0214,0.0214
CSG0BB, CSGCCBB, CSGCDBB, CSGDCBB, CSGDDBB = 0.886, 0.977, 0.399, 0.399, 0.0708
DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB = 0.2934, 0.90367, 0.3875, 0.3875, 0.0759

# Define all Function

In [37]:
def MLE_BB (data,AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB ,CSG0BB, CSGCCBB, CSGCDBB,CSGDCBB, CSGDDBB ,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB):
    for i in range(data.shape[0]):
       
        if data.iloc[i]['round'] == 1:
            if data.iloc[i]['coop'] == 'coop':
                data.loc[i, "AD"] = abs(ln(AD0BB))
                data.loc[i, "CSG"] = abs(ln(CSG0BB))
                data.loc[i, "DSG"] = abs(ln(DSG0BB))
            else:
                data.loc[i, "AD"] = abs(ln(1-AD0BB))
                data.loc[i, "CSG"] = abs(ln(1-CSG0BB))
                data.loc[i, "DSG"] = abs(ln(1-DSG0BB))
         
        
        else:
       
            if (data.iloc[i-1]['ocoop'] == "coop" and  data.iloc[i-1]['coop'] == "coop"):
                if  data.iloc[i]['coop'] == "coop":
                    data.loc[i, "AD"] = abs(ln(ADCCBB))
                    data.loc[i, "CSG"] = abs(ln(CSGCCBB))
                    data.loc[i, "DSG"] = abs(ln(DSGCCBB))
                else:
                    data.loc[i, "AD"] = abs(ln(1-ADCCBB))
                    data.loc[i, "CSG"] = abs(ln(1-CSGCCBB))
                    data.loc[i, "DSG"] = abs(ln(1-DSGCCBB))


            
            elif (data.iloc[i-1]['ocoop'] == "defect" and  data.iloc[i-1]['coop'] == "coop"):
                if  data.iloc[i]['coop'] == "coop":
                    data.loc[i, "AD"] = abs(ln(ADCDBB))
                    data.loc[i, "CSG"] = abs(ln(CSGCDBB))
                    data.loc[i, "DSG"] = abs(ln(DSGCDBB))

                else:
                    data.loc[i, "AD"] = abs(ln(1-ADCDBB))
                    data.loc[i, "CSG"] = abs(ln(1-CSGCDBB))
                    data.loc[i, "DSG"] = abs(ln(1-DSGCDBB))

                    
            elif (data.iloc[i-1]['ocoop'] == "coop" and  data.iloc[i-1]['coop'] == "defect"):
                if  data.iloc[i]['coop'] == "coop":
                    data.loc[i, "AD"] = abs(ln(ADDCBB))
                    data.loc[i, "CSG"] = abs(ln(CSGDCBB))
                    data.loc[i, "DSG"] = abs(ln(DSGDCBB))
                else:
                    data.loc[i, "AD"] = abs(ln(1-ADDCBB))
                    data.loc[i, "CSG"] = abs(ln(1-CSGDCBB))
                    data.loc[i, "DSG"] = abs(ln(1-DSGDCBB))

                    
            else:
                if (data.iloc[i-1]['ocoop'] == "defect" and  data.iloc[i-1]['coop'] == "defect"):
                    if  data.iloc[i]['coop'] == "coop":
                        data.loc[i, "AD"] = abs(ln(ADDDBB))
                        data.loc[i, "CSG"] = abs(ln(CSGDDBB))
                        data.loc[i, "DSG"] = abs(ln(DSGDDBB))

                    else:
                        data.loc[i, "AD"] = abs(ln(1-ADDDBB))
                        data.loc[i, "CSG"] = abs(ln(1-CSGDDBB))
                        data.loc[i, "DSG"] = abs(ln(1-DSGDDBB))

                        
    return(data)



In [38]:
def MLE_calculate_BB(data):
    table = pd.pivot_table(data, values=['AD',"CSG","DSG"], index=['ID'], aggfunc=np.sum)
    return (table)

In [39]:
def MLE_min_BB (data):
    mle = sum(data.min(axis=1))
    data['min'] = data.min(axis=1)
    return (mle,data)

In [40]:
def Cluster_BB (data):
    cluster = []
    for i in range (data.shape[0]):
        if data.iloc[i]['AD']== data.iloc[i]['min']:
            cluster.append(1)
        if data.iloc[i]['CSG']== data.iloc[i]['min']:
            cluster.append(2)
        if data.iloc[i]['DSG']== data.iloc[i]['min']:
            cluster.append(3)
    data['class'] = cluster
    return(data)



In [41]:
def vLookup_BB (data,table):
    data = pd.merge(data, table['class'], on ='ID', how ='inner',)
    return (data)



In [42]:
def N_class_BB (data):
    table2 = pd.pivot_table(data, values=["class"], index=['ID'], aggfunc=np.mean)
    
    class1 = sum(table2['class'] == 1)
    class2 = sum(table2['class'] == 2)
    class3 = sum(table2['class'] == 3)
    return(class1,class2,class3)



In [43]:
def vector_BB (data):
    table2 = pd.pivot_table(data, values=['roundone','aftercc','aftercd', 'afterdc', 'afterdd'],
                        index=["class"], aggfunc=np.mean)
    
    AD0, ADCC, ADCD, ADDC, ADDD = table2.iloc[0,4],table2.iloc[0,0],table2.iloc[0,1],table2.iloc[0,2],table2.iloc[0,3]
    CSG0, CSGCC, CSGCD, CSGDC, CSGDD =table2.iloc[1,4],table2.iloc[1,0],table2.iloc[1,1],table2.iloc[1,2],table2.iloc[1,3]
    DSG0, DSGCC, DSGCD, DSGDC, DSGDD = table2.iloc[2,4],table2.iloc[2,0],table2.iloc[2,1],table2.iloc[2,2],table2.iloc[2,3]
    
    table3 = pd.pivot_table(data, values=['roundone','aftercc','aftercd', 'afterdc', 'afterdd'],
                        index=["class"], aggfunc='count')
    
    AD0c, ADCCc, ADCDc, ADDCc, ADDDc = table3.iloc[0,4],table3.iloc[0,0],table3.iloc[0,1],table3.iloc[0,2],table3.iloc[0,3]
    CSG0c, CSGCCc, CSGCDc, CSGDCc, CSGDDc =table3.iloc[1,4],table3.iloc[1,0],table3.iloc[1,1],table3.iloc[1,2],table3.iloc[1,3]
    DSG0c, DSGCCc, DSGCDc, DSGDCc, DSGDDc = table3.iloc[2,4],table3.iloc[2,0],table3.iloc[2,1],table3.iloc[2,2],table3.iloc[2,3]
    
    AD0 = (((AD0*AD0c)+(ADCC*ADCCc)+(ADCD*ADCDc)+(ADDC*ADDCc)+(ADDD*ADDDc))/(AD0c+ADCCc+ADCDc+ADDCc+ADDDc))
    ADCC = ADCD = ADDC = ADDD = AD0
    
    CSGCD = (((CSGCDc * CSGCD)+(CSGDCc * CSGDC))/(CSGDCc+CSGCDc))
    CSGDC = CSGCD
    
    DSGCD = (((DSGCDc * DSGCD)+(DSGDCc * DSGDC))/(DSGDCc+DSGCDc))
    DSGDC = DSGCD
    
    return(AD0, ADCC, ADCD, ADDC, ADDD,CSG0, CSGCC, CSGCD, CSGDC, CSGDD,DSG0, DSGCC, DSGCD, DSGDC, DSGDD)

In [44]:
def change_class (df1,df2,fit=0):
    for i in range (df2.shape[0]):
        if df2.iloc[i]['class']== df1.iloc[i]['class']:
            fit+=1
    return (fit/1734)

In [45]:
def BB_Main_MLE (data,AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB ,CSG0BB, CSGCCBB, CSGCDBB, CSGDCBB, CSGDDBB ,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB):
    
    BBclass1 = []
    BBclass2 = []
    BBclass3 = []
    BBMLE = []
    class_change = []
    
    for i in range (1) :
        
        if i >0 :
            BB = MLE_BB (data,AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB ,CSG0BB, CSGCCBB, CSGCDBB,CSGDCBB, CSGDDBB ,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB)
            table_BB = MLE_calculate_BB (BB)
            BBmle , table_BB = MLE_min_BB(table_BB)
            df_cluster_BB = Cluster_BB(table_BB)
            fitt = change_class(df_cluster_BB2,df_cluster_BB,fit=0)
            df_vlook_BB = vLookup_BB (BB, table_BB)
            class1BB,class2BB,class3BB = N_class_BB (df_vlook_BB)
        
            
        else:
            BB = MLE_BB (data,AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB ,CSG0BB, CSGCCBB, CSGCDBB,CSGDCBB, CSGDDBB ,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB)
            table_BB = MLE_calculate_BB (BB)
            BBmle , table_BB = MLE_min_BB(table_BB)
            df_cluster_BB = Cluster_BB(table_BB)
            df_vlook_BB = vLookup_BB (BB, table_BB)
            class1BB,class2BB,class3BB = N_class_BB (df_vlook_BB)
            df_cluster_BB2 = df_cluster_BB
            fitt = 1
            
        
        AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB,CSG0BB, CSGCCBB, CSGCDBB, CSGDCBB, CSGDDBB,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB = vector_BB(df_vlook_BB) 
          
        
        class_change.append(fitt)
        BBMLE.append(BBmle)
        BBclass1.append(class1BB)
        BBclass2.append(class2BB)
        BBclass3.append(class3BB)
    
    
    return(BBMLE, BBclass1,BBclass2,BBclass3,class_change,AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB ,CSG0BB, CSGCCBB, CSGCDBB, CSGDCBB, CSGDDBB ,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB)



In [46]:
MLE_BB, class11_B,class22_B,class33_B,fit_class, AD0B, ADCCB, ADCDB, ADDCB, ADDDB ,CSG0B, CSGCCB, CSGCDB, CSGDCB, CSGDDB ,DSG0B, DSGCCB, DSGCDB, DSGDCB, DSGDDB = BB_Main_MLE(data,AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB ,CSG0BB, CSGCCBB, CSGCDBB, CSGDCBB, CSGDDBB ,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB)

In [47]:
MLE_BB

[43065.01783800889]